<a href="https://colab.research.google.com/github/omin-Kwon/real-time-squat-classifier/blob/main/model/SquatClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab Setup

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
"""
Change directory to where this file is located
"""
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


## Import Modules

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import activations
from tensorflow.keras.layers import AveragePooling2D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
from scipy import special
from sklearn.utils import shuffle

Gloabal parameters

In [8]:
data_path = './Pose_Dataset/'

image_size = 128
min_dim = 100
max_obj_num = 100

unet_base_filters = 64
in_channels = 3

w0 = 10
sigma = 5

batch_size = 16
learning_rate = 1e-4
num_epochs = 5

In [9]:
import json


labels=['bad_back_round','bad_back_warp','bad_head','bad_innner_thigh','bad_shallow','bad_toe','good']
paths={'bad_back_round':0,
       'bad_back_warp':0,
       'bad_head':0,
       'bad_innner_thigh':0,
       'bad_shallow':0,
       'bad_toe':0,
       'good':0}

for label in labels:
    paths[label]=data_path+label+"/1115_3djoints_index"


json_files={'bad_back_round':0,
       'bad_back_warp':0,
       'bad_head':0,
       'bad_innner_thigh':0,
       'bad_shallow':0,
       'bad_toe':0,
       'good':0}


for label in labels:
    json_files[label]=[pos_json for pos_json in os.listdir(paths[label]) if pos_json.endswith('.json')]

data={'bad_back_round':[],'bad_back_warp':[],'bad_head':[],'bad_innner_thigh':[],'bad_shallow':[],'bad_toe':[],'good':[]}

for label in labels:
    for single_file in json_files[label]:
        file_path=paths[label]+"/"+single_file
        with open(file_path,'r') as fp:
            data[label].append(json.load(fp))



In [10]:
data_numpy={}
for label in labels:
    label_list=[]
    for i in range(len(data[label])):
        my_list=[]
        for j in range(300):
            my_list.append(data[label][i][str(j)]['3d_joint'])
        label_list.append(my_list)
    label_numpy=np.array(label_list)
    data_numpy[label]=label_numpy

In [11]:
train_samples={}
valid_samples={}
test_samples={}
labels=['bad_back_round','bad_back_warp','bad_head','bad_innner_thigh','bad_shallow','bad_toe','good']
label_encoding={'bad_back_round':[1,0,0,0,0,0,0],'bad_back_warp':[0,1,0,0,0,0,0],'bad_head':[0,0,1,0,0,0,0],'bad_innner_thigh':[0,0,0,1,0,0,0],'bad_shallow':[0,0,0,0,1,0,0],'bad_toe':[0,0,0,0,0,1,0],'good':[0,0,0,0,0,0,1]}
train_labels=[]
valid_labels=[]
test_labels=[]
train_x=[]
valid_x=[]
test_x=[]

#train: valid: test = 5:3:2로 나눠봄
for label in labels:
    np.random.shuffle(data_numpy[label])
    num_train=int(0.5*len(data_numpy[label]))
    num_valid=int(0.8*len(data_numpy[label]))
    train_samples[label],valid_samples[label],test_samples[label]=data_numpy[label][:num_train,:],data_numpy[label][num_train:num_valid,:],data_numpy[label][num_valid:,:]

for label in labels:
    #print("----"+label+"-----")
    for i in range(train_samples[label].shape[0]):
        train_labels.append(label_encoding[label])
        train_x.append(train_samples[label][i,:,:].transpose())
    
    for i in range(valid_samples[label].shape[0]):
        valid_labels.append(label_encoding[label])
        valid_x.append(valid_samples[label][i,:,:].transpose())
    
    for i in range(test_samples[label].shape[0]):
        test_labels.append(label_encoding[label])   
        test_x.append(train_samples[label][i,:,:].transpose()) 

train_x=np.array(train_x)
train_labels=np.array(train_labels)

valid_x=np.array(valid_x)
valid_labels=np.array(valid_labels)
test_x=np.array(test_x)
test_labels=np.array(test_labels)

print("train_x",train_x.shape)
print("train_label",train_labels.shape)

print("test_x",test_x.shape)
print("test_label",test_labels.shape)

print("valid_x",valid_x.shape)
print("valid_label",valid_labels.shape)

train_x,train_labels=shuffle(train_x,train_labels)
valid_x,valid_labels=shuffle(valid_x,valid_labels)
test_x,test_labels=shuffle(test_x,test_labels)


train_x (1000, 171, 300)
train_label (1000, 7)
test_x (403, 171, 300)
test_label (403, 7)
valid_x (601, 171, 300)
valid_label (601, 7)


### Define ResnetBlock

In [12]:
def resblock(frames):
    #resblock에 들어온 input_layer
    input_layer=Input(shape=frames)
    
    res_layer=Conv1D(filters=256,kernel_size=1,strides=2)(input_layer)
    
    conv1d_layer1=Conv1D(filters=256,kernel_size=3,strides=2)(input_layer)
    conv1d_layer1=tf.keras.layers.ZeroPadding1D(padding=(0, 1))(conv1d_layer1)
    batch1=BatchNormalization()(conv1d_layer1)
    active1=activations.relu(batch1)

    conv1d_layer2=Conv1D(filters=256,kernel_size=3,strides=1,padding='same')(active1)
    batch2=BatchNormalization()(conv1d_layer2)
    active2=activations.relu(batch2)


    conv1d_layer3=Conv1D(filters=256,kernel_size=3,strides=1,padding='same')(active2)
    batch3=BatchNormalization()(conv1d_layer3)
    active3=activations.relu(batch3)
   
    output=tf.keras.layers.Add()([res_layer,active3])
    block=tf.keras.Model(inputs=input_layer,outputs=output)
    
    return block
    


### Define Model

In [13]:

points=[]
outputs=[]


inputs=tf.keras.Input(shape=(171,300,1))

# (B,171,300,1) matrix를 (B.300,1) matrix 171 개로 찢어준다.
for i in range(171):
    points.append(inputs[:,i,:,:])


for i in range(171):
    points[i]=Conv1D(filters=256,kernel_size=3,strides=2)(points[i])
    points[i]=ZeroPadding1D(padding=(0, 1))(points[i])
    points[i]=resblock((150,256))(points[i])
    points[i]=resblock((75,256))(points[i])
    points[i]=resblock((38,256))(points[i])
    points[i]=resblock((19,256))(points[i])

for i in range(171):
    points[i]=tf.expand_dims(points[i],axis=1)
    #print(points[i].shape)

point2D=tf.concat(points,axis=1)
#print(point2D.shape)


avg_pool_2d = AveragePooling2D(pool_size=(171, 10),  strides=(1, 1), padding='valid')

point2D=avg_pool_2d(point2D)
#print(point2D.shape)
point2D=Conv1D(filters=7,kernel_size=1,strides=1,activation='softmax')(point2D)
#print(point2D.shape)

point2D=point2D[:,0,0,:]
#print(point2D.shape)

my_model=tf.keras.Model(inputs=inputs,outputs=point2D)

In [ ]:
my_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
my_model.fit(x=train_x,y=train_labels,batch_size=10,epochs=30,validation_data=(valid_x,valid_labels))


"""Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)"""

Epoch 1/30
